<a href="https://colab.research.google.com/github/SanyaShresta25/Information-and-Network-Security/blob/main/Task%202/SecureKeyMngmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cryptography.hazmat.primitives.asymmetric.rsa as rsa
import cryptography.hazmat.primitives.asymmetric.dh as dh
import cryptography.hazmat.primitives.hashes as hashes
import cryptography.hazmat.primitives.kdf.pbkdf2 as pbkdf2
import cryptography.hazmat.primitives.serialization as serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

# Key Generation
def generate_symmetric_key():
    return os.urandom(32)  # AES-256 key

def generate_asymmetric_keys():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())
    public_key = private_key.public_key()
    return private_key, public_key

def generate_dh_parameters():
    return dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())

def generate_dh_keys(parameters):
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

def compute_dh_shared_key(private_key, peer_public_key):
    shared_key = private_key.exchange(peer_public_key)
    derived_key = pbkdf2.PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=os.urandom(16),
        iterations=100000,
        backend=default_backend()
    ).derive(shared_key)
    return derived_key

# Key Storage
def save_key_to_file(key, filename):
    with open(filename, 'wb') as f:
        f.write(key)

def load_key_from_file(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            return f.read()
    return None

def revoke_key(filename):
    if os.path.exists(filename):
        os.remove(filename)
        print(f"Key {filename} revoked.")
    else:
        print("Warning: Key does not exist.")

# Encryption and Decryption (AES)
def encrypt_with_aes(plaintext):
    key = load_key_from_file("aes_key.bin")
    if not key:
        print("Warning: AES key not found. Generate it first.")
        return

    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    padded_plaintext = plaintext.ljust(16 * ((len(plaintext) // 16) + 1), '\0')
    ciphertext = encryptor.update(padded_plaintext.encode()) + encryptor.finalize()
    save_key_to_file(iv + ciphertext, "encrypted_data.bin")
    print("Data encrypted and saved.")

def decrypt_with_aes():
    key = load_key_from_file("aes_key.bin")
    encrypted_data = load_key_from_file("encrypted_data.bin")

    if not key or not encrypted_data:
        print("Warning: AES key or encrypted data not found.")
        return

    iv, ciphertext = encrypted_data[:16], encrypted_data[16:]
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_text = decryptor.update(ciphertext) + decryptor.finalize()
    decrypted_text = decrypted_text.decode().rstrip('\0')
    print(f"Decrypted Text: {decrypted_text}")

# Operations
def generate_keys():
    aes_key = generate_symmetric_key()
    save_key_to_file(aes_key, "aes_key.bin")

    private_rsa, public_rsa = generate_asymmetric_keys()
    save_key_to_file(private_rsa.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    ), "private_rsa.pem")

    save_key_to_file(public_rsa.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ), "public_rsa.pem")

    print("Keys generated and saved.")

def perform_key_exchange():
    dh_params = generate_dh_parameters()
    private_dh, public_dh = generate_dh_keys(dh_params)
    peer_private_dh, peer_public_dh = generate_dh_keys(dh_params)
    shared_key = compute_dh_shared_key(private_dh, peer_public_dh)
    print(f"Shared Key: {shared_key.hex()}")

def revoke_aes_key():
    revoke_key("aes_key.bin")

def view_rsa_keys():
    private_key = load_key_from_file("private_rsa.pem")
    public_key = load_key_from_file("public_rsa.pem")

    if private_key and public_key:
        print("Stored Keys:")
        print(f"Private Key:\n{private_key.decode()}\n")
        print(f"Public Key:\n{public_key.decode()}\n")
    else:
        print("Warning: Keys not found.")

# Command-line interaction
if __name__ == "__main__":
    while True:
        print("\nSecure Key Management System")
        print("1. Generate Keys")
        print("2. Encrypt with AES")
        print("3. Decrypt with AES")
        print("4. Key Exchange")
        print("5. Revoke AES Key")
        print("6. View RSA Keys")
        print("7. Exit")

        choice = input("Choose an option: ")

        if choice == "1":
            generate_keys()
        elif choice == "2":
            plaintext = input("Enter text to encrypt: ")
            encrypt_with_aes(plaintext)
        elif choice == "3":
            decrypt_with_aes()
        elif choice == "4":
            perform_key_exchange()
        elif choice == "5":
            revoke_aes_key()
        elif choice == "6":
            view_rsa_keys()
        elif choice == "7":
            break
        else:
            print("Invalid choice. Please select a valid option.")



Secure Key Management System
1. Generate Keys
2. Encrypt with AES
3. Decrypt with AES
4. Key Exchange
5. Revoke AES Key
6. View RSA Keys
7. Exit
Choose an option: 1
Keys generated and saved.

Secure Key Management System
1. Generate Keys
2. Encrypt with AES
3. Decrypt with AES
4. Key Exchange
5. Revoke AES Key
6. View RSA Keys
7. Exit
Choose an option: 2
Enter text to encrypt: hello
Data encrypted and saved.

Secure Key Management System
1. Generate Keys
2. Encrypt with AES
3. Decrypt with AES
4. Key Exchange
5. Revoke AES Key
6. View RSA Keys
7. Exit
Choose an option: 3
Decrypted Text: hello

Secure Key Management System
1. Generate Keys
2. Encrypt with AES
3. Decrypt with AES
4. Key Exchange
5. Revoke AES Key
6. View RSA Keys
7. Exit
Choose an option: 4
Shared Key: 3c968cd3b110099e1f2bab85e9cc06b92eb384b4ff81811e60d7e19cd7982bda

Secure Key Management System
1. Generate Keys
2. Encrypt with AES
3. Decrypt with AES
4. Key Exchange
5. Revoke AES Key
6. View RSA Keys
7. Exit
Choose an 